# Weather Compensation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
import matplotlib.dates as mdates
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'notebook_connected'
from datetime import timedelta

In [2]:
df = pd.read_csv('simulation_results_finished.csv')
df['Date'] = pd.to_datetime(df['Date'])  # Convert 'Date' column to datetime
df.set_index('Date', inplace=True)
outdoor_temp_series = df['Dry-bulb temperature (°C)']
outdoor_temp_series = outdoor_temp_series.to_frame().reset_index()

# Convert time to seconds for simulation alignment instead of half-hourly intervals
outdoor_temp_series["time_seconds"] = (outdoor_temp_series["Date"] - outdoor_temp_series["Date"].iloc[0]).dt.total_seconds()


In [3]:
# Flow temperature calculation function
def calculate_target_flow_temp(lower_temp, upper_temp, lower_flow, upper_flow, outdoor_temp):
    """Heating Curve Calculation

    Args:
        lower_temp (float): lower outdoor temperature
        upper_temp (float): upper outdoor temperature
        lower_flow (float): lower flow temperature
        upper_flow (float): upper flow temperature
        outdoor_temp (float): outdoor temperature
    """
    if outdoor_temp < lower_temp:
        return upper_flow
    elif outdoor_temp > upper_temp:
        return lower_flow
    else:
        return lower_flow + ((upper_flow - lower_flow) / (upper_temp - lower_temp)) * (upper_temp - outdoor_temp)
    

# Define heating curve parameters
lower_outdoor_temp = -5  # Lower outdoor temperature for heating curve (°C)
upper_outdoor_temp = 20  # Upper outdoor temperature for heating curve (°C)
lower_flow_temp = 20     # Lower flow temperature for heating curve (°C)
upper_flow_temp = 70     # Upper flow temperature for heating curve (°C) 

In [4]:
# Simulation parameters
dt = 120  # time step in seconds
simulation_time = 24 * 3600 * 365  # simulate for 24 hours
timesteps = int(simulation_time / dt)

time = np.arange(0, simulation_time, dt)  # Time array for one year
# Interpolate outdoor temperature for simulation time steps
T_outdoor = np.interp(time, outdoor_temp_series["time_seconds"], outdoor_temp_series['Dry-bulb temperature (°C)'])

# Constants and Parameters from Table A2
C_PL = 2e6  # Thermal capacity of primary loop (J/K)
C_em = 2e6  # Thermal capacity of emitters (J/K)
C_int = 4e5  # Thermal capacity of indoor space (J/K)
C_env = 2.5e6  # Thermal capacity of building envelope (J/K)
cp_w = 4186  # Water specific heat capacity (J/kg/K)
m_dot = 0.05  # Primary loop mass flow rate (kg/s)
m_dot_SL = 0.02  # DHW secondary loop mass flow rate (kg/s)
m_dot_d_DHW = 0.002  # DHW demand flow rate (kg/s)

# Heat Transfer Coefficients
G_p_loss = 150  # Heat loss from primary loop (W/K)
G_em = 150.0  # Heat exchange from emitters to indoor space (W/K)
G_int = 95.0  # Heat loss from indoor space to envelope (W/K)
G_ext = 840.0  # Heat loss from envelope to ambient (W/K)
G_vent = 60.0  # Ventilation heat loss (W/K)
G_c_loss = 2  # Heat loss from cylinder (W/K)

# Cylinder Dimensions
Hc = 1.5  # Cylinder height (m)
Rc = 0.252  # Cylinder radius (m)
lambda_w = 0.6  # Water thermal conductivity (W/m/K)
Hc_top = 1.0  # Top section height (m)
Hc_bottom = 0.5  # Bottom section height (m)
m_total = np.pi * Rc**2 * Hc * 1000  # Total water mass (kg)

# Initial Conditions
T_PL = 45.0
T_em = 40.0
T_int = 18.0
T_env = 15.0
T_c_top = 50.0
T_c_bottom = 40.0
T_mains = 10.0

# Control and State Variables
# alpha_c_DHW = np.zeros(timesteps)
# beta_c_DHW = np.zeros(timesteps)
alpha_SH = 1
beta_SH = np.zeros(timesteps)
m_top = m_total * (2/3)
m_bottom = m_total * (1/3)

thermostat_temp = 20  # Thermostat set temperature in °C
hysteresis = 4.0  # Thermostat hysteresis in °C

# Outdoor Temperature
# T_outdoor = 10 + 5 * np.sin(np.linspace(0, 2 * np.pi, timesteps))

# Storage Arrays
T_PL_array = np.zeros(timesteps)
T_em_array = np.zeros(timesteps)
T_int_array = np.zeros(timesteps)
T_env_array = np.zeros(timesteps)
T_c_top_array = np.zeros(timesteps)
T_c_bottom_array = np.zeros(timesteps)
Q_HP_array = np.zeros(timesteps)

# Simulation Loop
for i in range(timesteps):  
    T_em_array[i] = T_em
    T_int_array[i] = T_int
    T_env_array[i] = T_env
    T_c_top_array[i] = T_c_top
    T_c_bottom_array[i] = T_c_bottom

    
    if T_outdoor[i] > 20:
        beta_SH[i] = 0
        T_PL = T_outdoor[i]     
    else:
        beta_SH[i] = 1
        T_PL = calculate_target_flow_temp(lower_outdoor_temp, upper_outdoor_temp, lower_flow_temp, upper_flow_temp, T_outdoor[i])
    T_PL_array[i] = T_PL

    # Thermal Dynamics Calculations
    # dT_PL_dt = (alpha_SH * G_p_loss * (T_em - T_PL) + (8500 if (beta_SH[i]) else 0)) / C_PL
    
    if beta_SH[i] == 0:
        Q_HP = 0
    else:
        if i == 0:
            dT_PL_dt = 0
        else:
            dT_PL_dt = (T_PL_array[i] - T_PL_array[i-1]) / dt
        Q_HP = dT_PL_dt * C_PL - alpha_SH * G_p_loss * (T_em - T_PL)
    Q_HP_array[i] = Q_HP

    dT_em_dt = (alpha_SH * G_p_loss * (T_PL - T_em) + G_em * (T_int - T_em)) / C_em

    dT_int_dt = (G_em * (T_em - T_int) + G_int * (T_env - T_int) + G_vent * (T_outdoor[i] - T_int)) / C_int

    dT_env_dt = (G_int * (T_int - T_env) + G_ext * (T_outdoor[i] - T_env)) / C_env


    # Update States
    # T_PL += dT_PL_dt * dt
    T_em += dT_em_dt * dt
    T_int += dT_int_dt * dt
    T_env += dT_env_dt * dt

In [5]:
# Save the results in a DataFrame
sim_results = pd.DataFrame({
    'Time': time,
    'Outdoor Temperature (°C)': T_outdoor,
    'Indoor Temperature (°C)': T_int_array,
    'Flow Temperature (°C)': T_PL_array,
    'Beta SH': beta_SH,
    'Emitter Temperature (°C)': T_em_array,
    'Heat Pump Heat Output (W)': Q_HP_array
})

In [6]:
sim_results.set_index('Time', inplace=True)
sim_results.index = pd.to_datetime(sim_results.index, unit='s', origin='unix').strftime('%m-%d %H:%M:%S')

In [10]:
df = sim_results

# Function to filter data for a specific day
def filter_data_for_day(df, day):
    start_time = pd.to_datetime(day)
    end_time = start_time + pd.Timedelta(days=1)
    return df[(df.index >= start_time.strftime('%m-%d %H:%M:%S')) & (df.index < end_time.strftime('%m-%d %H:%M:%S'))]

# Choose a specific day to display
specific_day = '2023-01-04 00:00:00'  # Change this to the desired date format 'YYYY-MM-DD HH:MM:SS'

daily_profile = filter_data_for_day(df, specific_day)

In [11]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=daily_profile.index, y=daily_profile['Indoor Temperature (°C)'], name='Indoor Temperature (°C)', line=dict(color='blue')),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=daily_profile.index, y=daily_profile['Flow Temperature (°C)'], name='Flow Temperature (°C)', line=dict(color='green')),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=daily_profile.index, y=daily_profile['Outdoor Temperature (°C)'], name='Outdoor Temperature (°C)', line=dict(color='orange')),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=daily_profile.index, y=daily_profile['Emitter Temperature (°C)'], name='Emitter Temperature (°C)', line=dict(color='yellow')),
    secondary_y=False,
)

# fig.add_trace(
#     go.Scatter(x=daily_profile.index, y=daily_profile['Cylinder Top Temperature (°C)'], name='Cylinder Top Temperature (°C)', line=dict(color='pink')),
#     secondary_y=False,
# )

# fig.add_trace(
#     go.Scatter(x=daily_profile.index, y=daily_profile['Alpha SH'], name='Alpha SH', line=dict(color='purple', dash='dot')),
#     secondary_y=True,
# )

fig.add_trace(
    go.Scatter(x=daily_profile.index, y=daily_profile['Heat Pump Heat Output (W)'], name='Heat Pump Heat Output (W)', line=dict(color='black', dash='dot')),
    secondary_y=True,
)

# Add thermostat bounds
fig.add_trace(go.Scatter(x=daily_profile.index, y=[thermostat_temp]*len(daily_profile.index), line=dict(color='red', dash='dash'), name="Thermostat Temperature (°C)"), secondary_y=False)
fig.add_trace(go.Scatter(x=daily_profile.index, y=[thermostat_temp - hysteresis / 2]*len(daily_profile.index), line=dict(color='grey', dash='dash'), name="Thermostat Lower Limit (°C)"), secondary_y=False)
fig.add_trace(go.Scatter(x=daily_profile.index, y=[thermostat_temp + hysteresis / 2]*len(daily_profile.index), line=dict(color='grey', dash='dash'), name="Thermostat Upper Limit (°C)"), secondary_y=False)


# Add figure title and axis labels
fig.update_layout(
    title_text=f"Temperature and heat power from {pd.to_datetime(specific_day).strftime('%m-%d %H:%M:%S')} to {(pd.to_datetime(specific_day) + timedelta(days=1)).strftime('%m-%d %H:%M:%S')}",
    width=1000,  # Set the width of the figure
    height=700,   # Set the height of the figure
    legend=dict(
        orientation="h",  # Horizontal orientation
        x=0.5,  # x position of the legend
        y=-0.5,  # y position of the legend
        xanchor="center",  # Anchor the legend at the center
        yanchor="bottom",  # Anchor the legend at the bottom
        bgcolor='rgba(255, 255, 255, 0.5)',  # Background color of the legend
        bordercolor='Black',  # Border color of the legend
        borderwidth=1,  # Border width of the legend
        itemwidth=30,  # Width of each legend item
        traceorder="normal"  # Order of legend items
    ),
    xaxis=dict(
        tickformat='%H:%M',  # Format ticks as hours and minutes
        dtick=3600/dt,  # Set tick step to 1 hour (3600000 milliseconds)
        tickangle=45  # Rotate x-axis labels for better readability
    ),
)
# Set x-axis title
fig.update_xaxes(title_text="Time (s)")

# Update y-axes
fig.update_yaxes(title_text='Temperature (°C)', secondary_y=False)
fig.update_yaxes(title_text='HP', secondary_y=True, tickfont=dict(color='black'), titlefont=dict(color='black'))

# Show plot
fig.show()
# pio.write_image(fig, f"./figures/Resistive heating hysteresis control from {pd.to_datetime(specific_day).strftime('%m-%d %H:%M:%S')} to {(pd.to_datetime(specific_day) + timedelta(days=1)).strftime('%m-%d %H:%M:%S')}.pdf")